In [36]:
import pandas as pd
from string import ascii_uppercase as alfabeto
import pickle

In [37]:
todas_tablas = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [3]:
# A -> H
# 11 -> 7*8 + 12 = 68
todas_tablas[26]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Argentina,0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Saudi Arabia,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Mexico,0,0,0,0,0,0,0,0,NaN
3,4,Poland,0,0,0,0,0,0,0,0,NaN


In [38]:
todas_tablas = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
dict_tablas = {}
for letra, i in zip(alfabeto, range(12,68,7)):
    df = todas_tablas[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True) # Cambiar nombre a una columna
    df.pop('Qualification') # Eliminar una columna
    dict_tablas[f'Grupo {letra}'] = df

In [6]:
dict_tablas['Grupo A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


In [39]:
with open('dict_table', 'wb') as output:
    pickle.dump(dict_tablas, output)

## Extraccion de partidos de futbol - Parte 1

In [34]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [35]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
       1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
       2018]

In [42]:
def get_matches(year): 
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    
    matches = soup.find_all('div', class_='footballbox')
    home = []
    score = []
    away = []
    
    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())
    
    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football

In [17]:
# historical data
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_wolrcup_historical_data.csv',index=False)

# fixture
df_fixture = get_matches('2022')
df_fixture.to_csv('fifa_wolrcup_fixture.csv',index=False)

## Extraccion de partidos de futbol - Parte 2

In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time


path = "C:\\Users\\MSI\\chromedriver-win64\\chromedriver.exe"
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

def obtener_data_faltante(year):
    
    web = f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"

    driver.get(web)

    matches = driver.find_elements(by='xpath', value='//tr[@itemprop="name"]')
    
    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find_element(by='xpath', value='./th[1]').text)
        score.append(match.find_element(by='xpath', value='./th[2]').text)
        away.append(match.find_element(by='xpath', value='./th[3]').text)
        
    dict_football = {'home':home, 'score':score, 'away':away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    time.sleep(2)
    return df_football

years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
       1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
       2018]

fifa = [obtener_data_faltante(year) for year in years]
driver.quit()

df_fifa = pd.concat(fifa, ignore_index=True)

df_fifa.to_csv('fifa_worldcup_missing_data.csv', index=False)